In [ ]:
!rm -rf *
!gdown --id 1nB_hn_ByoAP6hod3Nd7wlSf-43gPz31X
!unzip ml-1m.zip
!gdown --id 11xsjOjSV_oc9SXjD5CHh1bE8TwA81Tmv
!unzip data.zip


Downloading...
From: https://drive.google.com/uc?id=1nB_hn_ByoAP6hod3Nd7wlSf-43gPz31X
To: /content/ml-1m.zip
100% 5.92M/5.92M [00:00<00:00, 18.8MB/s]
Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
Downloading...
From: https://drive.google.com/uc?id=11xsjOjSV_oc9SXjD5CHh1bE8TwA81Tmv
To: /content/data.zip
100% 18.0M/18.0M [00:00<00:00, 43.5MB/s]
Archive:  data.zip
   creating: data/.ipynb_checkpoints/
  inflating: data/.ipynb_checkpoints/preprocess-1m-checkpoint.ipynb  
  inflating: data/.ipynb_checkpoints/preprocess-checkpoint.ipynb  
   creating: data/100k/
  inflating: data/100k/movie_cluster_100k.json  
  inflating: data/100k/movie_neighbor_100k.json  
  inflating: data/100k/movie_neighbor_100k_2.json  
  inflating: data/100k/test_100k.json  
  inflating: data/100k/train_100k.json  
  inflating: data/100k/user_cluster_100k.json  
  inflating

In [ ]:
import pandas as pd
import json
import numpy as np
import itertools

def json2df(path):
    data = json.load(open(path,"r"))
    df = pd.DataFrame.from_dict(data)
    df = df.rename(columns={'users': 'user_id', 'items': 'item_id'})
    df.user_id= df.user_id.astype(np.int64)
    df.item_id= df.item_id.astype(np.int64)
    df.rating = df.rating.astype(np.float32)
    return df


In [ ]:
def movie_preprocessing(movie):
    movie_col = list(movie.columns)
    movie_tag = [doc.split('|') for doc in movie['tag']]
    tag_table = {token: idx for idx, token in enumerate(set(itertools.chain.from_iterable(movie_tag)))}
    movie_tag = pd.DataFrame(movie_tag)
    tag_table = pd.DataFrame(tag_table.items())
    tag_table.columns = ['Tag', 'Index']

    genre_vecs = [];
    for i in range(len(movie)):
        genre_vec = []
        for j in range(len(tag_table)):
            if tag_table['Tag'][j] in list(movie_tag.iloc[i, :]):
                genre_vec.append(1)
            else:
                genre_vec.append(0);
        genre_vecs.append(genre_vec)
    
    movie['genre_vec'] = genre_vecs;
    movie = movie.drop('tag', 1)
    movie = movie.drop('item_name', 1)
    return movie


In [ ]:
 # read and preprocess the movie data
movies = pd.read_table('ml-1m/movies.dat', sep='::', names=['item_id', 'item_name', 'tag'], engine='python')
movies = movie_preprocessing(movies)
movies

,item_id,genre_vec
0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, ..."
1,2,"[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
2,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ..."
3,4,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, ..."
4,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
...,...,...
3878,3948,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
3879,3949,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3880,3950,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3881,3951,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
train = json2df("data/1m/train_1m.json")
test = json2df("data/1m/test_1m.json")

In [ ]:
train = train.merge(movies, how='inner', on='item_id')
test = test.merge(movies, how='inner', on='item_id')
dataset= train.append(test, ignore_index=True)
dataset

,user_id,item_id,rating,genre_vec
0,5972,2407,3.0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
1,3998,2407,1.0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
2,1074,2407,4.0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
3,380,2407,3.0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
4,2691,2407,4.0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
...,...,...,...,...
900183,4516,1420,5.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
900184,4088,874,3.0,"[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
900185,5795,591,1.0,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, ..."
900186,4169,701,4.0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
import keras
import tensorflow as tf
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate, Dropout, concatenate
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import numpy as np
n_users, n_movies = dataset.user_id.max(), dataset.item_id.max()
n_latent_factors = 12

In [ ]:
import keras.backend as K

def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

def newModel():

  user_input = keras.layers.Input(shape=[1],name='User')
  user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors,name='User-Embedding')(user_input))

  movie_input = keras.layers.Input(shape=[1],name='Item')
  movie_embedding = keras.layers.Embedding(n_movies + 1, n_latent_factors, name='Movie-Embedding')(movie_input)
  movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)

  genre_input = keras.layers.Input(shape=[len(train.genre_vec[0])],name='Genre')
  genre_embedding = keras.layers.Embedding(len(train.genre_vec[0]), 19, name='Genre-Embedding')(genre_input)
  genre_vec = keras.layers.Flatten(name='FlattenGenres')(genre_embedding)

  concat = concatenate([movie_vec, genre_vec])
  dense_1 = Dense(12, name ='Fully-Connected1', activation='relu')(concat)

  concat2=  concatenate([dense_1, user_vec])
  dense_2 = Dense(16, name ='Fully-Connected2', activation='relu')(concat2)
  dense_3 = Dense(12, name ='Fully-Connected3', activation='relu')(dense_2)
  prod = Dense(1, name ='Fully-Connected4', activation='relu')(dense_3)
  model = keras.Model([user_input, movie_input, genre_input], prod)

  model.compile(optimizer= 'adam', loss= rmse, metrics=['mean_absolute_error', 'mse', rmse])
  return(model)

In [ ]:
models=[]
weights = []
nfold = 4
from sklearn.model_selection import KFold
kfold = KFold(nfold, shuffle=True, random_state=1)
i=1
genre_test =  np.asarray([np.asarray(row, dtype=float) for row in test.genre_vec], dtype=float)
for train_index, test_index in kfold.split(train):
    print("Training model", i)
    ftrain = train.iloc[train_index]
    ftest = train.iloc[test_index]
    model = newModel()
    genreTrain = np.asarray([np.asarray(row, dtype=float) for row in ftrain.genre_vec], dtype=float)
    genreVal = np.asarray([np.asarray(row, dtype=float) for row in ftest.genre_vec], dtype=float)
    traindata = {
        "User": ftrain.user_id,
        "Item": ftrain.item_id,
        "Genre": genreTrain
    }
    valdata = {
        "User": ftest.user_id,
        "Item": ftest.item_id,
        "Genre": genreVal
    }
    history = model.fit(traindata, ftrain.rating, validation_data=(valdata, ftest.rating), epochs=10, verbose=1)

    score = model.evaluate(valdata, ftest.rating, batch_size=test.user_id.size, verbose=1)
    print("Train model", i,"completed. Test result: - loss:", score[0]," - mean_absolute_error: ", score[1]," - mean_square_error: ", score[2], " - rmse: ", score[3],  "\n")
    models.append(model)
    weights.append(score[3]);
    i+=1

Training model 1
Epoch 1/10
18754/18754 [==============================] - 55s 3ms/step - loss: 0.9364 - mean_absolute_error: 0.7501 - mse: 0.9049 - rmse: 0.9364 - val_loss: 0.8983 - val_mean_absolute_error: 0.7198 - val_mse: 0.8306 - val_rmse: 0.8983
Epoch 2/10
18754/18754 [==============================] - 58s 3ms/step - loss: 0.8953 - mean_absolute_error: 0.7118 - mse: 0.8164 - rmse: 0.8953 - val_loss: 0.8850 - val_mean_absolute_error: 0.7098 - val_mse: 0.8060 - val_rmse: 0.8849
Epoch 3/10
18754/18754 [==============================] - 57s 3ms/step - loss: 0.8809 - mean_absolute_error: 0.6988 - mse: 0.7907 - rmse: 0.8809 - val_loss: 0.8921 - val_mean_absolute_error: 0.7218 - val_mse: 0.8170 - val_rmse: 0.8921
Epoch 4/10
18754/18754 [==============================] - 57s 3ms/step - loss: 0.8713 - mean_absolute_error: 0.6906 - mse: 0.7737 - rmse: 0.8713 - val_loss: 0.8744 - val_mean_absolute_error: 0.6994 - val_mse: 0.7872 - val_rmse: 0.8744
Epoch 5/10
18754/18754 [===================

In [ ]:
print('Evaluate each model on test set')
i=1
testData =  {
  "User": test.user_id,
  "Item": test.item_id,
  "Genre": genre_test
}
for model in models:
  
    score= model.evaluate(testData, test.rating, batch_size=test.user_id.size, verbose=1)
    print("Model", i, ":", "-loss:", score[0], "mean_absolute_error:", score[1])
    i+=1

Evaluate each model on test set
1/1 [==============================] - 0s 116ms/step - loss: 0.8863 - mean_absolute_error: 0.6885 - mse: 0.7856 - rmse: 0.8863
Model 1 : -loss: 0.8863176107406616 mean_absolute_error: 0.6884663701057434
1/1 [==============================] - 0s 121ms/step - loss: 0.8859 - mean_absolute_error: 0.6952 - mse: 0.7848 - rmse: 0.8859
Model 2 : -loss: 0.8858759999275208 mean_absolute_error: 0.6951966881752014
1/1 [==============================] - 0s 115ms/step - loss: 0.9020 - mean_absolute_error: 0.7097 - mse: 0.8136 - rmse: 0.9020
Model 3 : -loss: 0.9019920825958252 mean_absolute_error: 0.7096909880638123
1/1 [==============================] - 0s 255ms/step - loss: 0.8846 - mean_absolute_error: 0.6915 - mse: 0.7824 - rmse: 0.8846
Model 4 : -loss: 0.8845571279525757 mean_absolute_error: 0.6914528608322144


In [ ]:
def mse(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.square(np.subtract(actual,pred)).mean() 
def mae(actual, pred):
    actual, pred = np.array(actual), np.array(pred)
    mae = np.mean(np.abs((actual - pred)))
    return mae

In [14]:
i = 0
pred = [0]*len(test)
sum = np.sum(weights)
weights = weights/sum

index = 0
for model in models:
  predict  = model.predict(testData)
  for i  in range (len(test)):
    pred[i] += weights[index] * predict[i][0]
  index+=1

In [15]:
print("Ensemble test result: mean_absolute_error=", mae(test['rating'], pred), "mean_square_error=", mse(test['rating'], pred), "rmse=", rmse(test['rating'], pred))

Ensemble test result: mean_absolute_error= 0.6855239370021786 mean_square_error= 0.7655640100685592 rmse= tf.Tensor(0.8749651479165094, shape=(), dtype=float64)
